In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# ------------------------------------------------------------------
# 1) Load the dataset
# ------------------------------------------------------------------
# 1) Load the data
df = pd.read_csv(
    "https://raw.githubusercontent.com/Ramil-cyber/Research_Linking_Analyzing_Deaths_US_Prisons/refs/heads/main/Data_Manipulation/Combined_death_pop.csv",
    low_memory=False,
)


# ------------------------------------------------------------------
# 2) Ensure numeric types
# ------------------------------------------------------------------
df["year"] = df["year"].astype(int)
for col in [
    "total_pop_15to64",
    "aapi_pct",
    "black_pct",
    "latinx_pct",
    "native_pct",
    "white_pct",
    "total_incarceration_rate",
    "female_total",
    "male_total",
    "deaths",
]:
    df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0)

# ------------------------------------------------------------------
# 3) Prepare years and state centroid mapping
# ------------------------------------------------------------------
years = sorted(df["year"].unique())
state_centroids = {
    "AL": {"lat": 32.8067, "lon": -86.7911},
    "AK": {"lat": 61.3707, "lon": -152.4044},
    "AZ": {"lat": 33.7298, "lon": -111.4312},
    "AR": {"lat": 34.9697, "lon": -92.3731},
    "CA": {"lat": 36.1162, "lon": -119.6816},
    "CO": {"lat": 39.0598, "lon": -105.3111},
    "CT": {"lat": 41.5978, "lon": -72.7554},
    "DE": {"lat": 39.3185, "lon": -75.5071},
    "DC": {"lat": 38.8974, "lon": -77.0268},
    "FL": {"lat": 27.7663, "lon": -81.6868},
    "GA": {"lat": 33.0406, "lon": -83.6431},
    "HI": {"lat": 21.0943, "lon": -157.4983},
    "ID": {"lat": 44.2405, "lon": -114.4788},
    "IL": {"lat": 40.3495, "lon": -88.9861},
    "IN": {"lat": 39.8494, "lon": -86.2583},
    "IA": {"lat": 42.0115, "lon": -93.2105},
    "KS": {"lat": 38.5266, "lon": -96.7265},
    "KY": {"lat": 37.6681, "lon": -84.6701},
    "LA": {"lat": 31.1695, "lon": -91.8678},
    "ME": {"lat": 44.6939, "lon": -69.3819},
    "MD": {"lat": 39.0639, "lon": -76.8021},
    "MA": {"lat": 42.2302, "lon": -71.5301},
    "MI": {"lat": 43.3266, "lon": -84.5361},
    "MN": {"lat": 45.6945, "lon": -93.9002},
    "MS": {"lat": 32.7416, "lon": -89.6787},
    "MO": {"lat": 38.4561, "lon": -92.2884},
    "MT": {"lat": 46.9219, "lon": -110.4544},
    "NE": {"lat": 41.1254, "lon": -98.2681},
    "NV": {"lat": 38.3135, "lon": -117.0554},
    "NH": {"lat": 43.4525, "lon": -71.5639},
    "NJ": {"lat": 40.2989, "lon": -74.5210},
    "NM": {"lat": 34.8405, "lon": -106.2485},
    "NY": {"lat": 42.1657, "lon": -74.9481},
    "NC": {"lat": 35.6301, "lon": -79.8064},
    "ND": {"lat": 47.5289, "lon": -99.7840},
    "OH": {"lat": 40.3888, "lon": -82.7649},
    "OK": {"lat": 35.5653, "lon": -96.9289},
    "OR": {"lat": 44.5720, "lon": -122.0709},
    "PA": {"lat": 40.5908, "lon": -77.2098},
    "RI": {"lat": 41.6809, "lon": -71.5118},
    "SC": {"lat": 33.8569, "lon": -80.9450},
    "SD": {"lat": 44.2998, "lon": -99.4388},
    "TN": {"lat": 35.7478, "lon": -86.6923},
    "TX": {"lat": 31.0545, "lon": -97.5635},
    "UT": {"lat": 40.1500, "lon": -111.8624},
    "VT": {"lat": 44.0459, "lon": -72.7107},
    "VA": {"lat": 37.7693, "lon": -78.1699},
    "WA": {"lat": 47.4009, "lon": -121.4905},
    "WV": {"lat": 38.4912, "lon": -80.9545},
    "WI": {"lat": 44.2685, "lon": -89.6165},
    "WY": {"lat": 42.7560, "lon": -107.3025},
}

# ------------------------------------------------------------------
# 4) Build choropleth + state labels for each year
# ------------------------------------------------------------------
fig = go.Figure()

for i, yr in enumerate(years):
    dff = df[df["year"] == yr]
    # Choropleth trace
    fig.add_trace(
        go.Choropleth(
            locations=dff["state_abbr"],
            locationmode="USA-states",
            z=dff["deaths"],
            zmin=df["deaths"].min(),
            zmax=df["deaths"].max(),
            colorscale="Reds",
            marker_line_color="white",
            customdata=dff[
                [
                    "total_pop_15to64",
                    "aapi_pct",
                    "black_pct",
                    "latinx_pct",
                    "native_pct",
                    "white_pct",
                    "total_incarceration_rate",
                    "female_total",
                    "male_total",
                ]
            ].values,
            hovertemplate=(
                "<b>%{location}</b><br>"
                + "Deaths: %{z}<br>"
                + "Pop15–64: %{customdata[0]:,}<br>"
                + "AAPI%: %{customdata[1]:.1f}<br>"
                + "Black%: %{customdata[2]:.1f}<br>"
                + "Latinx%: %{customdata[3]:.1f}<br>"
                + "Native%: %{customdata[4]:.1f}<br>"
                + "White%: %{customdata[5]:.1f}<br>"
                + "IncarcRate: %{customdata[6]:.1f}<br>"
                + "FemaleTotal: %{customdata[7]:,}<br>"
                + "MaleTotal: %{customdata[8]:,}<extra></extra>"
            ),
            visible=(i == 0),
        )
    )
    # Scattergeo for state_abbr
    lats = [state_centroids[s]["lat"] for s in dff["state_abbr"]]
    lons = [state_centroids[s]["lon"] for s in dff["state_abbr"]]
    fig.add_trace(
        go.Scattergeo(
            lon=lons,
            lat=lats,
            text=dff["state_abbr"],
            mode="text",
            showlegend=False,
            hoverinfo="none",
            textfont=dict(size=9, color="black"),
            visible=(i == 0),
        )
    )

# ------------------------------------------------------------------
# 5) Dropdown buttons for year selection
# ------------------------------------------------------------------
buttons = []
for i, yr in enumerate(years):
    # set visibility: two traces per year (choropleth + labels)
    vis = [False] * (len(years) * 2)
    vis[2 * i] = True
    vis[2 * i + 1] = True
    buttons.append(
        dict(
            label=str(yr),
            method="update",
            args=[{"visible": vis}, {"title": f"US Prisoner Deaths & Stats — {yr}"}],
        )
    )

fig.update_layout(
    title="US Prisoner Deaths & Stats — " + str(years[0]),
    geo=dict(scope="usa", lakecolor="white", bgcolor="#F0F0F0"),
    margin=dict(l=0, r=0, t=60, b=0),
    updatemenus=[dict(x=0.05, y=1.05, direction="down", buttons=buttons)],
)

# ------------------------------------------------------------------
# 6) Display the map
# ------------------------------------------------------------------
fig.write_html("interactive_prison_map.html", include_plotlyjs="cdn")
fig.show()

In [2]:
import pandas as pd
import plotly.graph_objects as go

# 1) Load the data
df = pd.read_csv(
    "https://raw.githubusercontent.com/Ramil-cyber/Research_Linking_Analyzing_Deaths_US_Prisons/refs/heads/main/Data_Manipulation/Combined_death_pop.csv",
    low_memory=False,
)

# ------------------------------------------------------------------
# Clean and cast types
# ------------------------------------------------------------------
df["year"] = df["year"].astype(int)
for col in [
    "deaths",
    "total_pop_15to64",
    "aapi_pct",
    "black_pct",
    "latinx_pct",
    "native_pct",
    "white_pct",
    "total_incarceration_rate",
    "female_total",
    "male_total",
]:
    df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0)

# ------------------------------------------------------------------
# Define state centroids (for label overlay)
# ------------------------------------------------------------------
state_centroids = {
    "AL": {"lat": 32.8067, "lon": -86.7911},
    "AK": {"lat": 61.3707, "lon": -152.4044},
    "AZ": {"lat": 33.7298, "lon": -111.4312},
    "AR": {"lat": 34.9697, "lon": -92.3731},
    "CA": {"lat": 36.1162, "lon": -119.6816},
    "CO": {"lat": 39.0598, "lon": -105.3111},
    "CT": {"lat": 41.5978, "lon": -72.7554},
    "DE": {"lat": 39.3185, "lon": -75.5071},
    "DC": {"lat": 38.8974, "lon": -77.0268},
    "FL": {"lat": 27.7663, "lon": -81.6868},
    "GA": {"lat": 33.0406, "lon": -83.6431},
    "HI": {"lat": 21.0943, "lon": -157.4983},
    "ID": {"lat": 44.2405, "lon": -114.4788},
    "IL": {"lat": 40.3495, "lon": -88.9861},
    "IN": {"lat": 39.8494, "lon": -86.2583},
    "IA": {"lat": 42.0115, "lon": -93.2105},
    "KS": {"lat": 38.5266, "lon": -96.7265},
    "KY": {"lat": 37.6681, "lon": -84.6701},
    "LA": {"lat": 31.1695, "lon": -91.8678},
    "ME": {"lat": 44.6939, "lon": -69.3819},
    "MD": {"lat": 39.0639, "lon": -76.8021},
    "MA": {"lat": 42.2302, "lon": -71.5301},
    "MI": {"lat": 43.3266, "lon": -84.5361},
    "MN": {"lat": 45.6945, "lon": -93.9002},
    "MS": {"lat": 32.7416, "lon": -89.6787},
    "MO": {"lat": 38.4561, "lon": -92.2884},
    "MT": {"lat": 46.9219, "lon": -110.4544},
    "NE": {"lat": 41.1254, "lon": -98.2681},
    "NV": {"lat": 38.3135, "lon": -117.0554},
    "NH": {"lat": 43.4525, "lon": -71.5639},
    "NJ": {"lat": 40.2989, "lon": -74.5210},
    "NM": {"lat": 34.8405, "lon": -106.2485},
    "NY": {"lat": 42.1657, "lon": -74.9481},
    "NC": {"lat": 35.6301, "lon": -79.8064},
    "ND": {"lat": 47.5289, "lon": -99.7840},
    "OH": {"lat": 40.3888, "lon": -82.7649},
    "OK": {"lat": 35.5653, "lon": -96.9289},
    "OR": {"lat": 44.5720, "lon": -122.0709},
    "PA": {"lat": 40.5908, "lon": -77.2098},
    "RI": {"lat": 41.6809, "lon": -71.5118},
    "SC": {"lat": 33.8569, "lon": -80.9450},
    "SD": {"lat": 44.2998, "lon": -99.4388},
    "TN": {"lat": 35.7478, "lon": -86.6923},
    "TX": {"lat": 31.0545, "lon": -97.5635},
    "UT": {"lat": 40.1500, "lon": -111.8624},
    "VT": {"lat": 44.0459, "lon": -72.7107},
    "VA": {"lat": 37.7693, "lon": -78.1699},
    "WA": {"lat": 47.4009, "lon": -121.4905},
    "WV": {"lat": 38.4912, "lon": -80.9545},
    "WI": {"lat": 44.2685, "lon": -89.6165},
    "WY": {"lat": 42.7560, "lon": -107.3025},
}

# Prepare lists of unique years & states
years = sorted(df["year"].unique())
states = sorted(df["state_abbr"].unique())

# Create figure with choropleth+labels per year
fig = go.Figure()
for i, yr in enumerate(years):
    dff = df[df["year"] == yr]
    # Choropleth trace
    fig.add_trace(
        go.Choropleth(
            locations=dff["state_abbr"],
            locationmode="USA-states",
            z=dff["deaths"],
            zmin=df["deaths"].min(),
            zmax=df["deaths"].max(),
            colorscale="Reds",
            customdata=dff[
                [
                    "total_pop_15to64",
                    "aapi_pct",
                    "black_pct",
                    "latinx_pct",
                    "native_pct",
                    "white_pct",
                    "total_incarceration_rate",
                    "female_total",
                    "male_total",
                ]
            ].values,
            hovertemplate=(
                "<b>%{location}</b><br>"
                "Deaths: %{z}<br>"
                "Pop15–64: %{customdata[0]:,}<br>"
                "AAPI%: %{customdata[1]:.1f}<br>"
                "Black%: %{customdata[2]:.1f}<br>"
                "Latinx%: %{customdata[3]:.1f}<br>"
                "Native%: %{customdata[4]:.1f}<br>"
                "White%: %{customdata[5]:.1f}<br>"
                "IncarcRate: %{customdata[6]:.1f}<br>"
                "FemaleTot: %{customdata[7]:,}<br>"
                "MaleTot: %{customdata[8]:,}<extra></extra>"
            ),
            visible=(i == 0),
        )
    )
    # Label trace
    lons = [state_centroids.get(s, {}).get("lon", None) for s in dff["state_abbr"]]
    lats = [state_centroids.get(s, {}).get("lat", None) for s in dff["state_abbr"]]
    fig.add_trace(
        go.Scattergeo(
            lon=lons,
            lat=lats,
            text=dff["state_abbr"],
            mode="text",
            showlegend=False,
            hoverinfo="none",
            visible=(i == 0),
            textfont=dict(size=9, color="black"),
        )
    )

# Year dropdown menu
year_buttons = []
for i, yr in enumerate(years):
    vis = [False] * (2 * len(years))
    vis[2 * i] = True
    vis[2 * i + 1] = True
    year_buttons.append(
        dict(
            label=str(yr),
            method="update",
            args=[{"visible": vis}, {"title": f"US Prisoner Deaths & Stats — {yr}"}],
        )
    )

# State dropdown menu
# Prepare z-values per state per year
z_by_state = {
    st: [
        df[(df["year"] == yr) & (df["state_abbr"] == st)]["deaths"].sum()
        for yr in years
    ]
    for st in states
}
chor_indices = list(range(0, 2 * len(years), 2))
state_buttons = []
for st in states:
    z_lists = [
        [
            z_by_state[st][yr_i] if loc == st else 0
            for loc in df[df["year"] == yr]["state_abbr"]
        ]
        for yr_i, yr in enumerate(years)
    ]
    state_buttons.append(
        dict(label=st, method="restyle", args=[{"z": z_lists}, chor_indices])
    )

# Attach menus
fig.update_layout(
    title=f"US Prisoner Deaths & Stats — {years[0]}",
    geo=dict(scope="usa", lakecolor="white", bgcolor="#F0F0F0"),
    margin=dict(l=0, r=0, t=80, b=0),
    updatemenus=[
        dict(x=0.05, y=1.05, direction="down", buttons=year_buttons, showactive=True),
        dict(x=0.25, y=1.05, direction="down", buttons=state_buttons, showactive=True),
    ],
)

# Render
fig.write_html("interactive_prison_map_state_year.html", include_plotlyjs="cdn")
fig.show()